# TSLA

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from scipy.stats import norm


tickers = [
    "NVDA", "AAPL", "MSFT", "AMZN", "GOOG", "GOOGL", 
    "MA", "META", "TSLA", "BRK-B"
]

# Tanggal awal dan akhir data
start_date = "2023-11-14"
end_date = "2024-11-14"

# Ambil data untuk semua ticker
all_data = {}
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    data['Log Return'] = np.log(data['Close'] / data['Close'].shift(1))
    all_data[ticker] = data

# Simpan hasil ke DataFrame gabungan untuk analisis lebih lanjut
returns_data = pd.DataFrame({ticker: all_data[ticker]['Log Return'] for ticker in tickers})

# Menghapus baris dengan NaN (awal setiap ticker memiliki 1 nilai NaN)
returns_data = returns_data.dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
# file_name = "log_returns.xlsx"
# returns_data.to_excel(file_name, sheet_name="Log Returns")

#### Menghitung Data Log Return

In [42]:
import yfinance as yf
import numpy as np
import math
from scipy.stats import norm


# Ambil data saham dari Yahoo Finance
ticker = "MSFT"  # Ganti dengan ticker saham yang diinginkan, contoh UNVR.JK
data = yf.download (ticker, start="2023-11-14", end ="2024-11-14")  # Periode yang sama seperti di dokumen

# Hitung return logaritmik harian
data['Log Return'] = np.log(data['Close'] / data['Close'].shift(1))
returns = data['Log Return'].dropna()  # Hilangkan NaN




[*********************100%***********************]  1 of 1 completed


In [2]:
returns

Date
2023-11-15    0.022614
2023-11-16   -0.038835
2023-11-17    0.003035
2023-11-20    0.005533
2023-11-21    0.023491
                ...   
2024-11-07    0.028630
2024-11-08    0.078697
2024-11-11    0.085807
2024-11-12   -0.063427
2024-11-13    0.005313
Name: Log Return, Length: 251, dtype: float64

In [97]:
ks_stat, p_value = kstest(returns, 'norm', args=(returns.mean(), returns.std()))

In [98]:
p_value

0.053145857032193766

#### Normalitas

In [99]:
from scipy.stats import jarque_bera
jb_stat, p_value = jarque_bera(returns)

print(f"Jarque-Bera Statistic: {jb_stat:.10f}")
print(f"P-value: {p_value:.20f}")

from scipy.stats import kstest
ks_stat, p_value = kstest(returns, 'norm')  # 'norm' untuk distribusi normal

# Tampilkan hasil dengan format yang diinginkan
print(f"Kolmogorov-Smirnov Statistic: {ks_stat:.10f}")
print(f"P-value: {p_value:.20f}")


Jarque-Bera Statistic: 233.8533615731
P-value: 0.00000000000000000000
Kolmogorov-Smirnov Statistic: 0.4554112672
P-value: 0.00000000000000000000


#### Nilai Opsi

In [43]:
S0 = 426.89  # Harga saham terakhir pada periode
K = 310 # Harga strike sebagai contoh, dapat disesuaikan
r = 0.04293  # Suku bunga bebas risiko sesuai dokumen (3.5%)
T = 8/252  # Waktu hingga jatuh tempo, misal 3 bulan atau 0.25 tahun
sigma = returns.std() * np.sqrt(252)


#### Black Sholes dan Gram Charlier

In [7]:
import numpy as np
from scipy.stats import norm

skewness = returns.skew()
kurtosis = returns.kurt()

# Perhitungan d1 dan d2 Black-Scholes
d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

# Harga opsi Black-Scholes (CBS)
CBS = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Koreksi harga opsi Gram-Charlier
Q3 = (1 / 6) * S0 * sigma * np.sqrt(T) * (norm.pdf(d1) * (2 * sigma * np.sqrt(T) - d1) + sigma**2 * T * norm.cdf(d1))
Q4 = (1 / 24) * S0 * sigma * np.sqrt(T) * (norm.pdf(d1) * (d1**2 - 3 * sigma * np.sqrt(T) * (d1 - sigma * np.sqrt(T)) - 1) + (sigma * np.sqrt(T))**3 * norm.cdf(d1))

# Harga opsi Gram-Charlier (CGC)
CGC = CBS + skewness * Q3 + (kurtosis - 3) * Q4

# Output hasil
print(f"Harga opsi Black-Scholes (CBS): {CBS:.10f}")
print(f"Harga opsi Gram-Charlier (CGC): {CGC:.10f}")


Harga opsi Black-Scholes (CBS): 216.3093832431
Harga opsi Gram-Charlier (CGC): 216.3568121537


#### Variance Gamma

In [44]:
import numpy as np
from scipy.stats import kurtosis, skew

def estimate_variance_gamma_params(log_returns, T):
    """
    Estimasi parameter Variance Gamma (sigma, theta, nu) menggunakan method of moments.
    
    Parameters:
        log_returns: array-like, log return data historis
        T: float, periode waktu (dalam tahun)

    Returns:
        sigma, theta, nu: float, parameter Variance Gamma
    """
    # Hitung momen data
    mean = np.mean(log_returns) / T
    variance = np.var(log_returns) / T
    skewness = skew(log_returns)
    kurt = kurtosis(log_returns, fisher=False)  # Fisher=False untuk kurtosis populasi

    # Estimasi parameter
    theta = skewness * (variance ** 1.5) / (2 * (kurt - 3))
    nu = (kurt - 3) * variance / (3 * (variance - theta**2))
    sigma = np.sqrt(variance - theta**2 * nu)

    return sigma, theta, nu



sigma, theta, nu = estimate_variance_gamma_params(returns, T)
print(f"sigma: {sigma:.4f}, theta: {theta:.4f}, nu: {nu:.4f}")


sigma: 0.0694, theta: -0.0001, nu: 0.8006


In [45]:
import numpy as np
from scipy.stats import norm

def variance_gamma_call_price(S0, K, T, r, sigma, theta, nu, n_simulations=10000):
    """
    Menghitung harga opsi call menggunakan model Variance Gamma dengan simulasi Monte Carlo.
    
    Parameters:
        S0: float, harga awal aset
        K: float, harga strike
        T: float, waktu hingga jatuh tempo
        r: float, tingkat bunga bebas risiko
        sigma: float, volatilitas dari proses Brownian
        theta: float, drift pada proses Brownian
        nu: float, tingkat varians dari perubahan waktu gamma
        n_simulations: int, jumlah simulasi Monte Carlo

    Returns:
        float, harga opsi call
    """
    # Simulasi distribusi gamma
    gamma = np.random.gamma(shape=T / nu, scale=nu, size=n_simulations)
    
    # Proses Variance Gamma
    # drift = (r - np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    # diffusion = theta * gamma + sigma * np.sqrt(gamma) * np.random.normal(size=n_simulations)
    # ST = S0 * np.exp(drift + diffusion)
    
    # Proses Variance Gamma
    drift = (r + (1/(nu))*np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    diffusion = theta * gamma + sigma * np.sqrt(gamma) * np.random.normal(size=n_simulations)
    ST = S0 * np.exp(drift + diffusion)
    
    # Nilai opsi call
    call_payoff = np.maximum(ST - K, 0)
    call_price = np.exp(-r * T) * np.mean(call_payoff)
    
    return call_price

# Contoh penggunaan
n_simulations = 100000  # Jumlah simulasi Monte Carlo

call_price = variance_gamma_call_price(S0, K, T, r, sigma, theta, nu, n_simulations)
print(f"Harga opsi call (Variance Gamma): {call_price:.4f}")


Harga opsi call (Variance Gamma): 117.3054


#### Antithetic Variate

In [38]:
import numpy as np

def variance_gamma_call_price_avr(S0, K, T, r, sigma, theta, nu, n_simulations=10000):
    """
    Menghitung harga opsi call menggunakan model Variance Gamma 
    dengan teknik antithetic variate reduction.
    
    Parameters:
        S0: float, harga awal aset
        K: float, harga strike
        T: float, waktu hingga jatuh tempo
        r: float, tingkat bunga bebas risiko
        sigma: float, volatilitas dari proses Brownian
        theta: float, drift pada proses Brownian
        nu: float, tingkat varians dari perubahan waktu gamma
        n_simulations: int, jumlah simulasi Monte Carlo

    Returns:
        float, harga opsi call
    """
    # Separuh jumlah simulasi untuk pasangan antithetic
    half_simulations = n_simulations 
    
    # Simulasi gamma
    gamma = np.random.gamma(shape=T / nu, scale=nu, size=half_simulations)
    
    # Simulasi normal untuk Z
    Z = np.random.normal(size=half_simulations)
    
    # Variance Gamma paths untuk pasangan (Z, -Z)
    drift = (r + (1 / nu) * np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    
    ST_plus = S0 * np.exp(drift + theta * gamma + sigma * np.sqrt(gamma) * Z)
    ST_minus = S0 * np.exp(drift + theta * gamma - sigma * np.sqrt(gamma) * Z)
    
    # Harga opsi call untuk pasangan antithetic
    payoff_plus = np.maximum(ST_plus - K, 0)
    payoff_minus = np.maximum(ST_minus - K, 0)
    
    # Estimasi harga opsi menggunakan rata-rata antithetic
    call_price = np.exp(-r * T) * np.mean((payoff_plus + payoff_minus) / 2)
    
    return call_price

# Contoh parameter

n_simulations = 100000  # Jumlah simulasi Monte Carlo

# Hitung harga opsi call dengan antithetic variate
call_price_avr = variance_gamma_call_price_avr(S0, K, T, r, sigma, theta, nu, n_simulations)
print(f"Harga opsi call (Antithetic Variate Reduction): {call_price_avr:.4f}")


Harga opsi call (Antithetic Variate Reduction): 142.9306


# BRK-B

#### Menghitung Data Log Return

In [58]:
import yfinance as yf
import numpy as np
import math
from scipy.stats import norm

# Fungsi Black-Scholes untuk menghitung harga opsi beli
def black_scholes_call(S0, K, r, T, sigma):
    # Hitung d1 dan d2
    d1 = (math.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    # Harga opsi beli menggunakan Black-Scholes
    call_price = S0 * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    return call_price

# Ambil data saham dari Yahoo Finance
ticker = "BRK-B"  # Ganti dengan ticker saham yang diinginkan, contoh UNVR.JK
data = yf.download (ticker, start="2023-11-14", end ="2024-11-14")  # Periode yang sama seperti di dokumen

# Hitung return logaritmik harian
data['Log Return'] = np.log(data['Close'] / data['Close'].shift(1))
returns = data['Log Return'].dropna()  # Hilangkan NaN





[*********************100%***********************]  1 of 1 completed


#### Normalitas

In [47]:
from scipy.stats import jarque_bera
jb_stat, p_value = jarque_bera(returns)

print(f"Jarque-Bera Statistic: {jb_stat:.10f}")
print(f"P-value: {p_value:.20f}")
from scipy.stats import kstest
ks_stat, p_value = kstest(returns, 'norm')  # 'norm' untuk distribusi normal

# Tampilkan hasil dengan format yang diinginkan
print(f"Kolmogorov-Smirnov Statistic: {ks_stat:.10f}")
print(f"P-value: {p_value:.20f}")




Jarque-Bera Statistic: 200.8372045803
P-value: 0.00000000000000000000
Kolmogorov-Smirnov Statistic: 0.4867448815
P-value: 0.00000000000000000000


#### Nilai Opsi

In [59]:
# print(f"Harga opsi beli (call option) dengan Black-Scholes adalah: {call_price_black_scholes:.4f}")
S0 = 467.7  # Harga saham terakhir pada periode
K = 457.7  # Harga strike sebagai contoh, dapat disesuaikan
r = 0.04293  # Suku bunga bebas risiko sesuai dokumen (3.5%)
T = 8/252  # Waktu hingga jatuh tempo, misal 3 bulan atau 0.25 tahun
sigma = returns.std() * np.sqrt(252)
#sigma = returns.std()

#### Black Sholes dan Gram Charlier

In [49]:
### Black Scholes dan Gram Charlier
import numpy as np
from scipy.stats import norm

skewness = returns.skew()
kurtosis = returns.kurt()

# Perhitungan d1 dan d2 Black-Scholes
d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

# Harga opsi Black-Scholes (CBS)
CBS = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Koreksi harga opsi Gram-Charlier
Q3 = (1 / 6) * S0 * sigma * np.sqrt(T) * (norm.pdf(d1) * (2 * sigma * np.sqrt(T) - d1) + sigma**2 * T * norm.cdf(d1))
Q4 = (1 / 24) * S0 * sigma * np.sqrt(T) * (norm.pdf(d1) * (d1**2 - 3 * sigma * np.sqrt(T) * (d1 - sigma * np.sqrt(T)) - 1) + (sigma * np.sqrt(T))**3 * norm.cdf(d1))

# Harga opsi Gram-Charlier (CGC)
CGC = CBS + skewness * Q3 + (kurtosis - 3) * Q4

# Output hasil
print(f"Harga opsi Black-Scholes (CBS): {CBS:.10f}")
print(f"Harga opsi Gram-Charlier (CGC): {CGC:.10f}")


Harga opsi Black-Scholes (CBS): 11.8100558414
Harga opsi Gram-Charlier (CGC): 11.6792694972


#### Variance Gamma

In [60]:

### Variance Gamma
import numpy as np
from scipy.stats import kurtosis, skew

def estimate_variance_gamma_params(log_returns, T):
    """
    Estimasi parameter Variance Gamma (sigma, theta, nu) menggunakan method of moments.
    
    Parameters:
        log_returns: array-like, log return data historis
        T: float, periode waktu (dalam tahun)

    Returns:
        sigma, theta, nu: float, parameter Variance Gamma
    """
    # Hitung momen data
    mean = np.mean(log_returns) / T
    variance = np.var(log_returns) / T
    skewness = skew(log_returns)
    kurt = kurtosis(log_returns, fisher=False)  # Fisher=False untuk kurtosis populasi

    # Estimasi parameter
    theta = skewness * (variance ** 1.5) / (2 * (kurt - 3))
    nu = (kurt - 3) * variance / (3 * (variance - theta**2))
    sigma = np.sqrt(variance - theta**2 * nu)

    return sigma, theta, nu



sigma, theta, nu = estimate_variance_gamma_params(returns, T)
print(f"sigma: {sigma:.4f}, theta: {theta:.4f}, nu: {nu:.4f}")



sigma: 0.0506, theta: 0.0000, nu: 1.4553


In [62]:
import numpy as np
from scipy.stats import norm

def variance_gamma_call_price(S0, K, T, r, sigma, theta, nu, n_simulations=10000):
    """
    Menghitung harga opsi call menggunakan model Variance Gamma dengan simulasi Monte Carlo.
    
    Parameters:
        S0: float, harga awal aset
        K: float, harga strike
        T: float, waktu hingga jatuh tempo
        r: float, tingkat bunga bebas risiko
        sigma: float, volatilitas dari proses Brownian
        theta: float, drift pada proses Brownian
        nu: float, tingkat varians dari perubahan waktu gamma
        n_simulations: int, jumlah simulasi Monte Carlo

    Returns:
        float, harga opsi call
    """
    # Simulasi distribusi gamma
    gamma = np.random.gamma(shape=T / nu, scale=nu, size=n_simulations)
    
    # Proses Variance Gamma
    # drift = (r - np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    # diffusion = theta * gamma + sigma * np.sqrt(gamma) * np.random.normal(size=n_simulations)
    # ST = S0 * np.exp(drift + diffusion)
    
    # Proses Variance Gamma
    drift = (r + (1/(nu))*np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    diffusion = theta * gamma + sigma * np.sqrt(gamma) * np.random.normal(size=n_simulations)
    ST = S0 * np.exp(drift + diffusion)
    
    # Nilai opsi call
    call_payoff = np.maximum(ST - K, 0)
    call_price = np.exp(-r * T) * np.mean(call_payoff)
    
    return call_price

# Contoh penggunaan
n_simulations = 100000  # Jumlah simulasi Monte Carlo

call_price = variance_gamma_call_price(S0, K, T, r, sigma, theta, nu, n_simulations)
print(f"Harga opsi call (Variance Gamma): {call_price:.4f}")



Harga opsi call (Variance Gamma): 10.7642


#### Antithetic Variate

In [63]:
import numpy as np

def variance_gamma_call_price_avr(S0, K, T, r, sigma, theta, nu, n_simulations=10000):
    """
    Menghitung harga opsi call menggunakan model Variance Gamma 
    dengan teknik antithetic variate reduction.
    
    Parameters:
        S0: float, harga awal aset
        K: float, harga strike
        T: float, waktu hingga jatuh tempo
        r: float, tingkat bunga bebas risiko
        sigma: float, volatilitas dari proses Brownian
        theta: float, drift pada proses Brownian
        nu: float, tingkat varians dari perubahan waktu gamma
        n_simulations: int, jumlah simulasi Monte Carlo

    Returns:
        float, harga opsi call
    """
    # Separuh jumlah simulasi untuk pasangan antithetic
    half_simulations = n_simulations 
    
    # Simulasi gamma
    gamma = np.random.gamma(shape=T / nu, scale=nu, size=half_simulations)
    
    # Simulasi normal untuk Z
    Z = np.random.normal(size=half_simulations)
    
    # Variance Gamma paths untuk pasangan (Z, -Z)
    drift = (r + (1 / nu) * np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    
    ST_plus = S0 * np.exp(drift + theta * gamma + sigma * np.sqrt(gamma) * Z)
    ST_minus = S0 * np.exp(drift + theta * gamma - sigma * np.sqrt(gamma) * Z)
    
    # Harga opsi call untuk pasangan antithetic
    payoff_plus = np.maximum(ST_plus - K, 0)
    payoff_minus = np.maximum(ST_minus - K, 0)
    
    # Estimasi harga opsi menggunakan rata-rata antithetic
    call_price = np.exp(-r * T) * np.mean((payoff_plus + payoff_minus) / 2)
    
    return call_price

# Contoh parameter

n_simulations = 100000  # Jumlah simulasi Monte Carlo

# Hitung harga opsi call dengan antithetic variate
call_price_avr = variance_gamma_call_price_avr(S0, K, T, r, sigma, theta, nu, n_simulations)
print(f"Harga opsi call (Antithetic Variate Reduction): {call_price_avr:.4f}")


Harga opsi call (Antithetic Variate Reduction): 10.7590


# META

#### Menghitung Data Log Return

In [52]:
import yfinance as yf
import numpy as np
import math
from scipy.stats import norm

# Fungsi Black-Scholes untuk menghitung harga opsi beli
def black_scholes_call(S0, K, r, T, sigma):
    # Hitung d1 dan d2
    d1 = (math.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    # Harga opsi beli menggunakan Black-Scholes
    call_price = S0 * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    return call_price

# Ambil data saham dari Yahoo Finance
ticker = "META"  # Ganti dengan ticker saham yang diinginkan, contoh UNVR.JK
data = yf.download (ticker, start="2023-11-14", end ="2024-11-14")  # Periode yang sama seperti di dokumen

# Hitung return logaritmik harian
data['Log Return'] = np.log(data['Close'] / data['Close'].shift(1))
returns = data['Log Return'].dropna()  # Hilangkan NaN





[*********************100%***********************]  1 of 1 completed


#### Normalitas

In [27]:
from scipy.stats import jarque_bera
jb_stat, p_value = jarque_bera(returns)

print(f"Jarque-Bera Statistic: {jb_stat:.10f}")
print(f"P-value: {p_value:.20f}")
from scipy.stats import kstest
ks_stat, p_value = kstest(returns, 'norm')  # 'norm' untuk distribusi normal

# Tampilkan hasil dengan format yang diinginkan
print(f"Kolmogorov-Smirnov Statistic: {ks_stat:.10f}")
print(f"P-value: {p_value:.20f}")




Jarque-Bera Statistic: 4176.3000165101
P-value: 0.00000000000000000000
Kolmogorov-Smirnov Statistic: 0.4732605996
P-value: 0.00000000000000000000


#### Nilai Opsi

In [54]:
# print(f"Harga opsi beli (call option) dengan Black-Scholes adalah: {call_price_black_scholes:.4f}")
S0 = 577.16  # Harga saham terakhir pada periode
K = 435  # Harga strike sebagai contoh, dapat disesuaikan
r = 0.04293  # Suku bunga bebas risiko sesuai dokumen (3.5%)
T = 8/252  # Waktu hingga jatuh tempo, misal 3 bulan atau 0.25 tahun
sigma = returns.std() * np.sqrt(252)
#sigma = returns.std()

#### Black Sholes dan Gram Charlier

In [29]:
### Black Scholes dan Gram Charlier
import numpy as np
from scipy.stats import norm

skewness = returns.skew()
kurtosis = returns.kurt()

# Perhitungan d1 dan d2 Black-Scholes
d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

# Harga opsi Black-Scholes (CBS)
CBS = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Koreksi harga opsi Gram-Charlier
Q3 = (1 / 6) * S0 * sigma * np.sqrt(T) * (norm.pdf(d1) * (2 * sigma * np.sqrt(T) - d1) + sigma**2 * T * norm.cdf(d1))
Q4 = (1 / 24) * S0 * sigma * np.sqrt(T) * (norm.pdf(d1) * (d1**2 - 3 * sigma * np.sqrt(T) * (d1 - sigma * np.sqrt(T)) - 1) + (sigma * np.sqrt(T))**3 * norm.cdf(d1))

# Harga opsi Gram-Charlier (CGC)
CGC = CBS + skewness * Q3 + (kurtosis - 3) * Q4

# Output hasil
print(f"Harga opsi Black-Scholes (CBS): {CBS:.10f}")
print(f"Harga opsi Gram-Charlier (CGC): {CGC:.10f}")


Harga opsi Black-Scholes (CBS): 142.8499476103
Harga opsi Gram-Charlier (CGC): 142.9001345371


#### Variance Gamma

In [55]:

### Variance Gamma
import numpy as np
from scipy.stats import kurtosis, skew

def estimate_variance_gamma_params(log_returns, T):
    """
    Estimasi parameter Variance Gamma (sigma, theta, nu) menggunakan method of moments.
    
    Parameters:
        log_returns: array-like, log return data historis
        T: float, periode waktu (dalam tahun)

    Returns:
        sigma, theta, nu: float, parameter Variance Gamma
    """
    # Hitung momen data
    mean = np.mean(log_returns) / T
    variance = np.var(log_returns) / T
    skewness = skew(log_returns)
    kurt = kurtosis(log_returns, fisher=False)  # Fisher=False untuk kurtosis populasi

    # Estimasi parameter
    theta = skewness * (variance ** 1.5) / (2 * (kurt - 3))
    nu = (kurt - 3) * variance / (3 * (variance - theta**2))
    sigma = np.sqrt(variance - theta**2 * nu)

    return sigma, theta, nu



sigma, theta, nu = estimate_variance_gamma_params(returns, T)
print(f"sigma: {sigma:.4f}, theta: {theta:.4f}, nu: {nu:.4f}")



sigma: 0.1244, theta: 0.0001, nu: 6.5756


In [56]:
import numpy as np
from scipy.stats import norm

def variance_gamma_call_price(S0, K, T, r, sigma, theta, nu, n_simulations=10000):
    """
    Menghitung harga opsi call menggunakan model Variance Gamma dengan simulasi Monte Carlo.
    
    Parameters:
        S0: float, harga awal aset
        K: float, harga strike
        T: float, waktu hingga jatuh tempo
        r: float, tingkat bunga bebas risiko
        sigma: float, volatilitas dari proses Brownian
        theta: float, drift pada proses Brownian
        nu: float, tingkat varians dari perubahan waktu gamma
        n_simulations: int, jumlah simulasi Monte Carlo

    Returns:
        float, harga opsi call
    """
    # Simulasi distribusi gamma
    gamma = np.random.gamma(shape=T / nu, scale=nu, size=n_simulations)
    
    # Proses Variance Gamma
    # drift = (r - np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    # diffusion = theta * gamma + sigma * np.sqrt(gamma) * np.random.normal(size=n_simulations)
    # ST = S0 * np.exp(drift + diffusion)
    
    # Proses Variance Gamma
    drift = (r + (1/(nu))*np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    diffusion = theta * gamma + sigma * np.sqrt(gamma) * np.random.normal(size=n_simulations)
    ST = S0 * np.exp(drift + diffusion)
    
    # Nilai opsi call
    call_payoff = np.maximum(ST - K, 0)
    call_price = np.exp(-r * T) * np.mean(call_payoff)
    
    return call_price

# Contoh penggunaan
n_simulations = 100000  # Jumlah simulasi Monte Carlo

call_price = variance_gamma_call_price(S0, K, T, r, sigma, theta, nu, n_simulations)
print(f"Harga opsi call (Variance Gamma): {call_price:.4f}")



Harga opsi call (Variance Gamma): 142.7900


#### Antithetic Variate

In [57]:
import numpy as np

def variance_gamma_call_price_avr(S0, K, T, r, sigma, theta, nu, n_simulations=10000):
    """
    Menghitung harga opsi call menggunakan model Variance Gamma 
    dengan teknik antithetic variate reduction.
    
    Parameters:
        S0: float, harga awal aset
        K: float, harga strike
        T: float, waktu hingga jatuh tempo
        r: float, tingkat bunga bebas risiko
        sigma: float, volatilitas dari proses Brownian
        theta: float, drift pada proses Brownian
        nu: float, tingkat varians dari perubahan waktu gamma
        n_simulations: int, jumlah simulasi Monte Carlo

    Returns:
        float, harga opsi call
    """
    # Separuh jumlah simulasi untuk pasangan antithetic
    half_simulations = n_simulations 
    
    # Simulasi gamma
    gamma = np.random.gamma(shape=T / nu, scale=nu, size=half_simulations)
    
    # Simulasi normal untuk Z
    Z = np.random.normal(size=half_simulations)
    
    # Variance Gamma paths untuk pasangan (Z, -Z)
    drift = (r + (1 / nu) * np.log(1 - theta * nu - 0.5 * sigma**2 * nu)) * T
    
    ST_plus = S0 * np.exp(drift + theta * gamma + sigma * np.sqrt(gamma) * Z)
    ST_minus = S0 * np.exp(drift + theta * gamma - sigma * np.sqrt(gamma) * Z)
    
    # Harga opsi call untuk pasangan antithetic
    payoff_plus = np.maximum(ST_plus - K, 0)
    payoff_minus = np.maximum(ST_minus - K, 0)
    
    # Estimasi harga opsi menggunakan rata-rata antithetic
    call_price = np.exp(-r * T) * np.mean((payoff_plus + payoff_minus) / 2)
    
    return call_price

# Contoh parameter

n_simulations = 100000  # Jumlah simulasi Monte Carlo

# Hitung harga opsi call dengan antithetic variate
call_price_avr = variance_gamma_call_price_avr(S0, K, T, r, sigma, theta, nu, n_simulations)
print(f"Harga opsi call (Antithetic Variate Reduction): {call_price_avr:.4f}")


Harga opsi call (Antithetic Variate Reduction): 142.7867
